In [ ]:
dataiku_env = True
try:
    import dataiku
except:
    dataiku_env = False

In [ ]:
from os import listdir
from os.path import join, isfile
from lxml import etree
#import src.data
from src.data import PackageManager
#from src.data import XbrlConst
from src.data import xbrl2rdf, process_linkbase, process_instance, process_schema
from src.data.FileSource import openFileSource
from collections import defaultdict
from io import StringIO, BytesIO
import rdflib
import numpy as np

In [ ]:
if dataiku_env:
    handle = dataiku.Folder("taxonomies")
    XBRL_TAXONOMY_PATH = handle.get_path()
    output_dir = XBRL_TAXONOMY_PATH + "/processed"
else:
    XBRL_TAXONOMY_PATH = join('..', 'data', 'external', 'xbrl', 'taxonomies')
    output_dir = join('..', 'data', 'rdf', 'xbrl', 'taxonomies')

In [ ]:
taxonomies = [f for f in listdir(XBRL_TAXONOMY_PATH) if isfile(join(XBRL_TAXONOMY_PATH, f)) and f[-3:]=='zip']

manager = PackageManager.Taxonomies(XBRL_TAXONOMY_PATH)

for taxonomy in taxonomies:
    manager.addPackage(join(XBRL_TAXONOMY_PATH, taxonomy))
manager.rebuildRemappings()
manager.save()

In [ ]:
for item in manager.config['packages']:
    print(item['URL'])

In [ ]:
f = openFileSource(manager.config['packages'][2]['URL'])
f.mappedPaths = manager.config['packages'][1]["remappings"]

package_name = manager.config['packages'][2]['name']
package_uri = manager.config['packages'][2]['URL']

In [ ]:
xmlfiles = [file for file in f.dir if (file[-3:] in ['xsd', 'xml'])]
print(len(xmlfiles))

In [ ]:
roots = list()

for file in xmlfiles:
    fp = f.fs.open(file, "r")
    content = fp.read()
    roots.append((file, etree.fromstring(content, parser=etree.XMLParser(remove_comments=True))))

In [ ]:
namespaces = {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
              'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
              'xl': 'http://www.xbrl.org/2003/XLink',
              'xsd': 'http://www.w3.org/2001/XMLSchema',
              'ext': 'http://eiopa.europa.eu/xbrl/ext',
              'xbrll': 'https://w3id.org/vocab/xbrll',
              's2c_dom': 'http://eiopa.europa.eu/xbrl/s2c/dict/dom'}
                          
for idx, root in enumerate(roots):
    ns = root[1].nsmap.copy()
    ns = dict((k.lower(), v.lower()) for k,v in ns.items() if (k is not None) and (v is not None))
    namespaces = {**namespaces, **root[1].nsmap}
if None in namespaces.keys():
    del namespaces[None]

In [ ]:
def process_files(process_function, package_name, extension, output_file):
    print('Processing of {} started'.format(output_file))
    g = rdflib.Graph()

    params = {'linkNumber' : 0, 'namespaces': namespaces}

    prefix = '# RDF triples (in turtle syntax) imported from XBRL resource\n'
    prefix += '# URI:  '+package_uri+'\n'
    prefix += '# name: '+package_name+'\n\n'
    prefix += '# the namespaces\n'
    for namespace in namespaces.keys():
        if '#' in namespaces[namespace]:
            prefix += "@prefix "+namespace.lower()+": <"+namespaces[namespace].lower()+">.\n"
        else:
            prefix += "@prefix "+namespace.lower()+": <"+namespaces[namespace].lower()+"/>.\n"
    prefix += '\n'

    s = prefix
    for idx, root in enumerate(roots):
        print('Progress: {}% done'.format(np.round(100 * idx/len(roots),decimals=1)), end='\r')
        params['output'] = StringIO()
        params['base'] = "http://"+"/".join(root[0].split("/")[1:-1])
        if root[0][-3:]==extension:
            params = process_function(root[1], params)
        string = params['output'].getvalue().replace('\u2264', '')
        try:
            g.parse(data=prefix+string, format='turtle')
            s += string
        except:
            print("error in: " +str(idx) + ": "+ root[0])
            print(string)
            
    fp = open(join(output_dir, output_file), "w", encoding = "utf-8")
    fp.write(s)
    fp.close()

In [ ]:
process_files(process_schema.processSchema, package_name, 'xsd', "schema.ttl")

In [ ]:
process_files(process_linkbase.processLinkBase, package_name, 'xml', "linkbase.ttl")